algoritmo que: 
* a gestão de filas de espera em hospitais envolve a classificação de risco e priorização dos pacientes;
* monitorar a quantidade de pessoas inscritas e o tempo de espera;
* definir a prioridade de atendimento. Cada cor tem um tempo de espera tolerável, que indica a gravidade do caso:
    * Vermelho: risco de morte, atendimento imediato;
     * Laranja: tempo de espera tolerável de até 10 minutos;
    * Amarelo: tempo de espera tolerável de até 20 minutos;
    * Verde: não há risco de morte, tempo de espera tolerável de até 60 minutos;
    * Azul: quadros crônicos, encaminhamento ao centro de saúde, tempo de espera tolerável de até 180 minutos.
* organize a agenda de pacientes, devendo considerar:
    * o número de consultas marcadas;
    * o tempo médio de duração e a previsão de contratempos.
* a capacitação dos profissionais de saúde influencia no tamanho das filas.

Em observação à lei abaixo, esta será considerada na implementação do algoritmo:
* A Lei 10.048/00 garante atendimento prioritário a idosos com 60 anos ou mais na administração pública.
* Também inclui pessoas do seguinte perfil: deficientes, gestantes, lactantes, pessoas com crianças de colo e obesos.
* Todavia, indivíduos com idade 80+ tem preferência sob todos os demais.

In [1]:
# !pip install faker
from faker import Faker
import random

import time

import pandas as pd
import datetime

### Gerando dados para teste

In [3]:
# Inicializando o Faker e configurando para nomes em português
fake = Faker("pt_BR")

# Inicializando as listas

lista_nomes = []          # string: primeiro nome
lista_dt_nascimento = []  # date: a data de nascimento
lista_prioridade = []     # bool: False para NÃO; True para SIM
lista_dt_entrada = []     # datetime, indicando o horário de cadastro
lista_especialidade = []  # string: especialidade médica (geral, neurologia, oftalmologista etc)
lista_tma = []            # int: minutos com o tempo médio de atendimento para cada especialidade
lista_class_risco = []    # string: vermelho, laranja, amarelo, verde, azul

lista_colunas = ['Nome', 'Nascimento', 'Prioridade', 'Entrada', 'Especialidade', 'TMA', 'Risco']
df = pd.DataFrame(columns=lista_colunas)

# Opções para os campos
especialidades_exemplo = ["geral", "neurologia", "oftalmologia", "cardiologia", "dermatologia", "pediatria"]
class_risco_exemplo = ["vermelho", "laranja", "amarelo", "verde", "azul"]

# Função para gerar uma data de nascimento aleatória
def gerar_data_nascimento():
    start_date = datetime.date(1940, 1, 1)
    end_date = datetime.date(2005, 12, 31)
    return start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))

# Função para gerar uma data de entrada aleatória com horário
def gerar_data_entrada():
    now = datetime.datetime.now()
    delta_days = random.randint(0, 5)
    delta_seconds = random.randint(0, 86400)  # Número de segundos no dia
    return now - datetime.timedelta(days=delta_days, seconds=delta_seconds)

# Populando as listas com 100 registros
for _ in range(100):
    lista_nomes.append(fake.first_name())  # Gera um primeiro nome aleatório em português
    lista_dt_nascimento.append(gerar_data_nascimento())
    lista_prioridade.append(random.choice([True, False]))
    lista_dt_entrada.append(gerar_data_entrada())
    lista_especialidade.append(random.choice(especialidades_exemplo))
    lista_tma.append(random.choice([15, 30, 45, 60]))  # Tempo médio de atendimento entre 10 e 60 minutos
    lista_class_risco.append(random.choice(class_risco_exemplo))
    
lista_dados = [lista_nomes, lista_dt_nascimento, lista_prioridade, lista_dt_entrada,
               lista_especialidade, lista_tma, lista_class_risco]
    
for i, coluna in enumerate(lista_colunas):
    df[coluna] = lista_dados[i]
    
df

,Nome,Nascimento,Prioridade,Entrada,Especialidade,TMA,Risco
0,Caleb,1962-02-26,False,2024-10-26 14:03:24.867562,neurologia,30,vermelho
1,Maria Fernanda,1991-08-06,True,2024-10-26 03:17:06.867562,oftalmologia,30,azul
2,Rafael,1947-02-02,False,2024-10-26 01:36:05.867562,neurologia,15,verde
3,Mathias,1961-09-26,False,2024-10-23 12:11:14.867562,oftalmologia,30,azul
4,Antônio,1996-01-29,False,2024-10-23 03:12:50.867562,dermatologia,30,amarelo
...,...,...,...,...,...,...,...
95,Mariah,1953-08-12,True,2024-10-28 07:38:17.870613,geral,45,azul
96,Manuela,1954-09-25,True,2024-10-24 10:17:35.870613,pediatria,30,laranja
97,Guilherme,2003-09-10,True,2024-10-27 16:15:41.870613,cardiologia,60,verde
98,Julia,1947-08-14,True,2024-10-27 17:37:56.870613,dermatologia,45,vermelho


### Iniciando funções e classes

In [22]:
class Fila():
    def __init__(self):
        self.data = []

    def inserir(self, x):
        self.data.append(x)

    def remover(self):
        if len(self.data) > 0:
            return self.data.pop(0)

    def topo(self):
        if len(self.data) > 0:
            return self.data[0]

    def vazio(self):
        return not len(self.data) > 0
    
    def quantidade(self):
        return len(self.data)

    def exibir(self):
        return list(self.data)
    
    
    
def calc_idade(nascido):
    """
    Calcula a idade de um cliente com base na data de nascimento fornecida.

    A função calcula a idade atual do cliente subtraindo o ano de nascimento do
    ano atual. Ajusta a idade caso o mês e o dia de nascimento ainda não tenham
    ocorrido no ano atual.

    Parâmetros:
    - nascido (datetime.date): A data de nascimento do cliente.

    Retorna:
    - int: A idade do usuário em anos.
    """
    
    now = datetime.datetime.now()
    return now.year - nascido.year - ((now.month, now.day) < (nascido.month, nascido.day))

Há duas, principais, alternativas: 
1. Trabalhar com uma única fila, e dentro desta operar inserções e remanejos conforme necessário para adequar aos três graus de prioridades.
2. Fazer uso de três filas, uma para cada prioridade, mais intuitivo e fácil de implementar.

Neste primeiro momento, acabarei optando pela segunda alternativa.

In [24]:
# iniciando cada objeto fila

fila_normal = Fila()
fila_priori1 = Fila()
fila_priori2 = Fila()

In [25]:
# simulando a inserção de dados em cada fila

for row in df.iterrows():    
    user = row[1].copy()
    idade = calc_idade(user['Nascimento'])
    user['Idade'] = idade
    senha = str(user['Especialidade'][:5]+'-'+str(row[0])).upper()
    if idade >= 80:        
        user['Senha'] = 'PE-'+senha    # PE > Prioridade Extra
        fila_priori2.inserir(user)
    elif idade >= 60 or user.Prioridade:
        user['Senha'] = 'P-'+senha
        fila_priori1.inserir(user)
    else: fila_normal.inserir(user)

In [26]:
fila_normal.quantidade()

31

In [27]:
fila_priori1.quantidade()

61

In [28]:
fila_priori2.quantidade()

8

In [ ]:
# ponderar sobre as cores em contraste com as prioridades

#### A se melhorar
* evitar dados duplicados em instâncias de objeto Fila()
* melhor geração de senha para guichê

fontes:
    https://www.camara.leg.br/noticias/630767-PESSOAS-MAIS-IDOSAS-PODERAO-TER-PRIORIDADE-NO-ATENDIMENTO